In [67]:
#loading Libraries
import dask.dataframe as dd
import pandas as pd

In [68]:
#2
# Load the CSV file into a Dask dataframe
csv_file = "timeseries.csv"
df = dd.read_csv(csv_file, dtype={'lat': 'float64', 'long': 'float64', 'cases': 'float64', 'deaths': 'float64', 'population': 'float64'}, parse_dates=['date'])

In [81]:
#3.1 Create a new data frame object that consists of samples (i.e., rows) corresponding to states in the US.
# Convert date column to datetime format
df["date"] = dd.to_datetime(df["date"])
# Filter data to include only rows corresponding to states in the US
df = df[(df['country'] == 'United States') & (df['state'].notnull())]


In [83]:
#3.2 During the time period 2020-Jan-01 to 2021-Feb-28, rank states in terms of their per-capita mortality? Compute per-capita mortality during a specific period as the ratio of total deaths during that time period, to the average population of the state (compute the average population during the time period).
# Filter data within the time period 2020-Jan-01 to 2021-Feb-28
df = df[(df['date'] >= "2020-01-01") & (df['date'] <= "2021-02-28")]
df_grouped = df.groupby('state').agg({'deaths': 'sum', 'population': 'mean'})
df_grouped['per_capita_mortality'] = df_grouped['deaths'] / df_grouped['population']
df_ranked = df_grouped.compute().sort_values(by='per_capita_mortality', ascending=False)
df_ranked

,deaths,population,per_capita_mortality
state,,,
New York,3852431.0,6.023470e+05,6.395701
Michigan,880814.0,2.748462e+05,3.204753
Louisiana,423430.0,1.548027e+05,2.735288
Illinois,765763.0,3.746646e+05,2.043863
New Jersey,1710535.0,8.421302e+05,2.031200
Georgia,309920.0,1.529650e+05,2.026085
Pennsylvania,758770.0,4.142725e+05,1.831572
Virginia,186685.0,1.373630e+05,1.359063
Mississippi,104608.0,7.710448e+04,1.356705


In [85]:
#3.3 During the same time period, compute the case fatality rate (CFR) per month, This computation should yield an array of dimensions 50 (states) X 14 (months). State the assumptions you are making in computing this metric.
# Compute Case Fatality Rate (CFR) per month
df['month'] = df['date'].dt.to_period('M')
df_monthly = df.groupby(['state', 'month']).agg({'cases': 'sum', 'deaths': 'sum'})
df_monthly['CFR'] = df_monthly['deaths'] / df_monthly['cases']
df_cfr_matrix = df_monthly.reset_index().compute().pivot(index='state', columns='month', values='CFR')
df_cfr_matrix

month,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07
state,,,,,,,
Alabama,NaN,NaN,0.005323,0.028309,0.038893,0.029629,0.023818
Alaska,NaN,NaN,0.003350,0.023145,0.021969,0.013032,0.012074
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,0.0,0.000000,0.000000,0.014865,0.019922,0.002115,0.009735
Arkansas,NaN,NaN,0.009157,0.019115,0.021296,0.015152,0.012744
California,0.0,0.000000,0.020067,0.034800,0.039834,0.031787,0.025430
Colorado,NaN,NaN,0.009393,0.026366,0.053720,0.054192,0.050637
Connecticut,NaN,NaN,0.018148,0.064776,0.090162,0.093831,0.095008
Delaware,NaN,NaN,0.013341,0.027340,0.035748,0.041647,0.040131


In [87]:
#3.4 Using this matrix as input, compute the ranking of states, on how the CFR rate changed over time. This computation will involve an aggregation of of month-to-month changes in CFR. Note that some of these month-to-month changes can be positive (CFR increases from a previous month to the current month) or negative (CFR decrease from a previous month to the current month), or zero (no change in CFR). You need to aggregate these individual changes, across all time periods.
# Compute month-to-month CFR changes
df_cfr_changes = df_cfr_matrix.diff(axis=1)
df_cfr_aggregated = df_cfr_changes.sum(axis=1).sort_values(ascending=False)
df_cfr_aggregated

state
New Jersey                      0.078497
Connecticut                     0.076860
Massachusetts                   0.074380
Pennsylvania                    0.063375
New Hampshire                   0.061680
Michigan                        0.061298
Rhode Island                    0.057596
New York                        0.051587
Illinois                        0.048028
Maryland                        0.043883
Colorado                        0.041244
Minnesota                       0.039581
Washington                      0.039217
Washington, D.C.                0.038932
Ohio                            0.037465
New Mexico                      0.032985
West Virginia                   0.030047
Indiana                         0.029179
Missouri                        0.027825
Delaware                        0.026790
Maine                           0.026276
California                      0.025430
Wisconsin                       0.024958
Mississippi                     0.022385
Oregon    